In [1]:
import pickle
import gseapy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
from cmcrameri import cm
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests

In [2]:
## June 11, redo with new miRNA target list from Sneha
# Previously, pathway enrichment (overrepresentation) was done using consensusDB. Website was not working
# Using gseapy's enrichR api (as previously done with per-module pathway enrichment)


In [3]:
class Cohort:
  def __init__(self, name):
    self.name = name

data_dir = r'C:\Users\neil_\DellXPS_June2024\OneDrive\Desktop\New UHN\Precision Medicine\carotid_plaque\data'

In [4]:
# With overrepresentation, we should consider what out background set is. Normally, gseapy uses all genes in selected pathway database
# We should define our background set as all possible genes that could have been included
# With the proteomics analysis, there are ~1500 proteins/genes tested.
# The miRNA analysis complicates things, as a DE-miRNA list was put through target enrichment (MIENTURNET). The results exapnd the number
# of possible genes (must consider all possible target genes for background set, not just the ones <FDR)


# load background set for DEP analysis. This will be the same for each cohort
# Background set for DE-miRNA will be cohort specific
with open(data_dir+"/genes_background_set.txt", 'r') as file:
    lines = file.readlines()
bg_genes_prot = set([line.strip() for line in lines])

In [5]:
Asymp = Cohort("Asymptomatic")

with open(data_dir+'/as_mirna_targets.pkl', 'rb') as file:
    Asymp.mirna_target_map = pickle.load(file)

# List of all targeted genes
Asymp.mirna_targets = list(set([gene for gene_list in list(Asymp.mirna_target_map.values()) for gene in gene_list]))
Asymp.mirnas = list(Asymp.mirna_target_map.keys())

with open(data_dir+"/deps_asymptomatic.txt", 'r') as file:
    lines = file.readlines()
deps = [line.strip() for line in lines]
Asymp.proteins = deps

# Get background gene list, use all miRNA targets from MEINTURNET
file_name = '/Mienturnet Enrichment results miRTarBase_Asymptomatic_PlaquevsMarginal_June2024.xlsx'
df = pd.read_excel(data_dir+file_name, header=0, skiprows=[0])
bg_genes_mir = set(df["Gene Symbol"])
Asymp.bg_set = bg_genes_prot.union(bg_genes_mir)


Asymp.gseapy = {}

In [6]:
Symp = Cohort("Symptomatic")

with open(data_dir+'/as_mirna_targets.pkl', 'rb') as file:
    Symp.mirna_target_map = pickle.load(file)

# List of all targeted genes
Symp.mirna_targets = list(set([gene for gene_list in list(Symp.mirna_target_map.values()) for gene in gene_list]))
Symp.mirnas = list(Symp.mirna_target_map.keys())

with open(data_dir+"/deps_symptomatic.txt", 'r') as file:
    lines = file.readlines()
deps = [line.strip() for line in lines]
Symp.proteins = deps

# Get background gene list, use all miRNA targets from MEINTURNET
file_name = '/Mienturnet Enrichment results miRTarBase_Symptomatic_PlaquevsMarginal.xlsx'
df = pd.read_excel(data_dir+file_name, header=0, skiprows=[0])
bg_genes_mir = set(df["Gene Symbol"])
Symp.bg_set = bg_genes_prot.union(bg_genes_mir)

Symp.gseapy = {}

In [7]:
def read_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Module enrichment analysis using Fishers exact
def fisher_bh(modset, de_list, N, q_value_threshold):
    modsize_dict = {module: len(members) for module, members in modset.items() if len(members) > 1}
    p_values = {}
    for module in modsize_dict.keys():
        members = modset[module]
        a = len([member for member in de_list if member in members])
        b = len([member for member in de_list if member not in members])
        c = len(members) - a
        d = N - (a + b + c)
        _, p_value = fisher_exact([[a, b], [c, d]])
        p_values[module] = p_value

    p_adjusted = multipletests(list(p_values.values()), method='fdr_bh')[1]
    return {module: p_val for module, p_val in zip(p_values.keys(), p_adjusted) if p_val < q_value_threshold}

def filter_modset(modset, list_modules):
    filtered_modset = {mod:mems for mod,mems in modset.items() if mod in list_modules}
    return filtered_modset

In [8]:

modset = read_json(data_dir+"/mm_louvain.json")

for Cohort in [Symp, Asymp]:
    de_list = Cohort.mirnas + Cohort.proteins
    enr_mod_results =fisher_bh(modset, de_list, 29079, 0.05) #background set size should be all prots/genes in experiment, but used N nodes in network for now
    Cohort.module_enrichment = enr_mod_results
    module_list = list(enr_mod_results.keys())
    Cohort.enriched_modset = filter_modset(modset, module_list)


In [9]:
# Now need to map proteins to genes. 

protein_nodes = set()

for Cohort in [Symp, Asymp]:
    for members in Cohort.enriched_modset.values():
        # Add items from each list to the set
        prot_members = [m for m in members if not m.startswith("hsa")]
        protein_nodes.update(prot_members)

len(protein_nodes)

21427

In [10]:
import os
import time
from UniProtMapper import ProtMapper

def read_pkl(path_pkl):
    '''Reads a .pkl file and returns the loaded object.'''
    if os.path.exists(path_pkl):
        with open(path_pkl, 'rb') as file:
            obj = pickle.load(file)
        return obj
    else:
        print(f".pkl not found: {path_pkl}")
        return None


def save_pkl(obj, path_save, overwrite=False):
    '''Save a python object as a .pkl file.'''
    
    # Ensure saved file is a .pkl
    root, ext = os.path.splitext(path_save)
    if ext != ".pkl":
        path_save += ".pkl"
    
    
    # Check if .pkl actually exists
    bool_fileExists =  os.path.exists(path_save)
    
    # Set overwrite to false if file doesnt exist
    if bool_fileExists == False:
        overwrite = False
    
    else:
        if overwrite==False:
            print(f"File already exists: {path_save}")
            print("Set arg: overwrite=True to overwrite.")
            print(".pkl not saved")
            return 

    # Save/overwrite file
    with open(path_save, "wb") as file:
        pickle.dump(obj, file)
            
        print(f"File {'overwritten' if overwrite else 'saved'}: {path_save}")

def read_mapping_dict(path_mapping):
    '''Read previously saved mapping dict from a .pkl file.'''
    file_type = os.path.splitext(path_mapping)[1]
    if file_type != ".pkl":
        print("Can only read .pkl files for mapping dict")
    else:
        dict_mapping = read_pkl(path_mapping)
        return dict_mapping


def create_prot_gene_map(set_proteinsToMap):
    '''Map a set of proteins to genes using the ProtMapper.'''
    mapper = ProtMapper()
    dict_mapping = {}
    set_unmappable = set()
    list_proteinsToMap = list(set_proteinsToMap)
    n = 0

    while n < len(list_proteinsToMap):
        batch = list_proteinsToMap[n: n + 500]
        try:
            mapped, unmapped = mapper.get(
                ids=batch, from_db="UniProtKB_AC-ID", to_db="Gene_Name"
            )
            dict_mappingBatch = dict(zip(mapped["From"], mapped["To"]))
            dict_mapping.update(dict_mappingBatch)
            set_unmappable.update(unmapped)
        except Exception as e:
            print("Error in mapping process:", e)
            # Optionally, handle the error, e.g., retry or log

        time.sleep(5)
        n += 500

    # Include unmappable proteins with None as their mapping
    for protein in set_unmappable:
        dict_mapping[protein] = None

    return dict_mapping


def update_mapping(dict_mapping, proteins):
    '''Update existing mapping dict'''
    set_proteins = set(proteins)
    set_mappedProteins = set(dict_mapping.keys())
    set_UnmappedProteins = set_proteins.difference(set_mappedProteins)
    
    if set_UnmappedProteins:
        dict_mappingNew = create_prot_gene_map(set_UnmappedProteins)
        
    if dict_mappingNew:
        dict_mapping.update(dict_mappingNew)
            
    return dict_mapping



def update_mapping_file(path_mapping, set_proteins):
    '''Overwrites mapping dict file (e.g mapping_dict.pkl)'''
    dict_mapping = read_mapping_dict(path_mapping)
    if dict_mapping is None:
        dict_mapping = {}
    dict_mappingUpdated = update_mapping(dict_mapping, set_proteins)
    save_pkl(dict_mappingUpdated, path_mapping, overwrite=True)

def get_umappable(dict_mapping):
    unmappable = [p for p,g in dict_mapping.items() if g is None]
    return set(unmappable)

In [11]:
uniprot_gene_map = read_pkl(r"C:\Users\neil_\DellXPS_June2024\OneDrive\Desktop\New UHN\Precision Medicine\wang_lab\id_mapping\uniprot_gene_map.pkl")

In [12]:
def module_as_genes(module, protein_map, mirna_map):
    '''Convert all protein/miRNA nodes in module to genes/gene targets'''
    gene_module = set()
    for mem in module:
        if mem.startswith("hsa"):
            genes = mirna_map.get(mem, []) #default to empty list if mem is not found
            if genes is None:
                genes = []
        else:
            gene = protein_map.get(mem)
            if gene is None:
                continue #skip protein member if no mapping
            genes = [gene]

        gene_module.update(genes)

    return gene_module

def modset_as_gene_modset(modset, protein_map, mirna_map):
    '''Convert modules in a module set to gene modules'''
    gene_modset={}
    for mod,mems in modset.items():
        gene_mems = module_as_genes(mems, protein_map, mirna_map)
        gene_modset[mod] = gene_mems

    return gene_modset


In [14]:
protein_map = uniprot_gene_map
for Cohort in [Symp, Asymp]:
    sig_modset = Cohort.enriched_modset
    mirna_map = Cohort.mirna_target_map #This is specific to Cohort
    
    Cohort.enriched_gene_modset = modset_as_gene_modset(sig_modset, protein_map, mirna_map)


In [15]:
# Initialize an empty set to store the combined result
master_bg = set()

for Cohort in [Symp, Asymp]:
    
    # Iterate over the values in the dictionary and update the combined set
    for value_set in Cohort.enriched_gene_modset.values():
        master_bg.update(value_set)
    master_bg.update(Cohort.bg_set)




In [16]:
def enrichr_module(gene_members, db, bg_set,fdr=0.05):
    g = gseapy.enrichr(gene_list=list(gene_members),
                       gene_sets=db,
                       organism='human',
                       background=bg_set)
    df = g.results
    df = df[df["Adjusted P-value"] < fdr].copy()
    df['a'] = df["Overlap"].str.split("/").str[0].astype(int)
    df['b'] = len(gene_members)
    df['c'] = df["Overlap"].str.split("/").str[1].astype(int)
    df['d'] = len(bg_set)
    df['Fold_Enrichment'] = (df['a']/ df['b']) / (df['c'] /df['d'])
    df.drop(columns=['a','b','c','d'],inplace=True)
    return df

In [17]:
def enrichr_per_module(gene_modset, db, bg_set, fdr=0.05):
    results_per_module = []
    for mod, gene_mems in gene_modset.items():
        if len(gene_mems)==0:
            continue
        df = enrichr_module(gene_mems, db=db, bg_set=bg_set, fdr=fdr)
        df["module"] = mod
        results_per_module.append(df)
    compiled_df = pd.concat(results_per_module)
    return compiled_df

In [18]:
## KEGG PER MODULE
bg_set = master_bg
db = "KEGG_2021_Human"
fdr = 0.05
for Cohort in [Symp, Asymp]:
    gene_modset = Cohort.enriched_gene_modset
    Cohort.gseapy[db] = enrichr_per_module(gene_modset, db, bg_set, fdr)

C:\Users\neil_\anaconda3\envs\uhn\Lib\site-packages\gseapy\enrichr.py:427: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results.append(res, ignore_index=True)
C:\Users\neil_\anaconda3\envs\uhn\Lib\site-packages\gseapy\enrichr.py:427: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results.append(res, ignore_index=True)
C:\Users\neil_\anaconda3\envs\uhn\Lib\site-packages\gseapy\enrichr.py:427: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results.append(res, ignore_index=True)
C:\Users\neil_\anaconda3\envs\uhn\Lib\site-packages\gseapy\enrichr.py:427: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.c